# Plotting with Matplotlib and Cartopy

In addition to supporting the HoloViz ecosystem of plotting packages via the `.plot()` accessors, UXarray also provides functionality to represent unstructured grids in formats that are accepted by Matplotlib and Cartopy.

This guide covers:
1. Rasterizing Data onto a Cartopy {class}`~cartopy.mpl.geoaxes.GeoAxes`
2. Visualizing Data with {class}`~matplotlib.collections.PolyCollection`
3. Visualizing Grid Topology

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from cartopy.crs import PlateCarree

import uxarray as ux

In [ ]:
base_path = "../../test/meshfiles/mpas/QU/480/"
grid_path = base_path + "grid.nc"
data_path = base_path + "data.nc"

uxds = ux.open_dataset(grid_path, data_path)

## Matplotlib and Cartopy Background

To support Matplotlib and Cartopy workflows, UXarray has chosen to provide the necessary conversion functions to represent unstructured grids in formats that can be interpreted by these packages. This means that you as the user are responsible for setting up the figure, adding colorbar, and configuring other aspects of the plotting workflow. Because of this, we will not cover these topics in detail, but recommend reviewing the following resources:
- [**Pythia Foundations:** Matplotlib](https://foundations.projectpythia.org/core/matplotlib)
- [**Pythia Foundations:** Cartopy](https://foundations.projectpythia.org/core/cartopy)
- [Matplotlib Quickstart Guide](https://matplotlib.org/stable/users/explain/quick_start.html)


## Rasterization

UXarray can rapidly translate face-centered data into a raster image that can be displayed directly on a Cartopy {class}`~cartopy.mpl.geoaxes.GeoAxes`.


UXarray currently supports a nearest-neighbor based rasterization method, which converts each screen-space pixel from the {class}`~cartopy.mpl.geoaxes.GeoAxes` into a geographic coordinate for sampling the underlying unstructured grid. If the pixel lies within a face in the unstructured grid, it is shaded by the corresponding face value.

The result is a 2-D array that works seamlessly with Matplotlib's `imshow`, `contour`, `contourf` and other visualization functions.


```{important}
Since the core geometry routines used internally directly sample the underlying unstructured grid using Numba, rasterization is extremely fast, even on high-resolution unstructured grids.
```

### Displaying Rasterized Data with `ax.imshow()`

Because rasterization yields a fully georeferenced two-dimensional array, the quickest way to render it is with Matplotlib's {meth}`~cartopy.mpl.geoaxes.GeoAxes.imshow` on a Cartopy {class}`~cartopy.mpl.geoaxes.GeoAxes`. By supplying the raster array along with the appropriate origin and extent parameters, Cartopy automatically handles projection and alignment.

```{caution}
When rasterizing a grid at a global extent, especially for higher-resolution grids, there may not be enough pixels to sample the entire grid thoroughly with the default `pixel_ratio` of 1.0. You can consider increasing the `pixel_ratio` if you need more pixels. The impact is demonstrated in an example below.
```

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.Robinson()}, figsize=(9, 6), constrained_layout=True
)

ax.set_global()

raster = uxds["bottomDepth"].to_raster(ax=ax)
img = ax.imshow(
    raster, cmap="Blues", origin="lower", extent=ax.get_xlim() + ax.get_ylim()
)
ax.set_title("Global Raster")
ax.coastlines()

# Adding a colorbar (the examples below will not include one to keep things concise)
cbar = fig.colorbar(img, ax=ax, fraction=0.03)

When you only need to visualize a subset of your data, such as a country, basin, or smaller study area, limiting the extent of the Cartopy {class}`~cartopy.mpl.geoaxes.GeoAxes` before rasterization can significantly improve performance. By setting a tighter longitude-latitude window, the pixel-to-face lookups are constrained to that region, reducing the overall number of queries. This targeted sampling speeds up rendering, lowers memory overhead, and produces a cleaner, more focused map of your area of interest.

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.Robinson()}, figsize=(9, 6), constrained_layout=True
)

ax.set_extent((-20, 20, -10, 10))


raster = uxds["bottomDepth"].to_raster(ax=ax)
ax.imshow(raster, cmap="Blues", origin="lower", extent=ax.get_xlim() + ax.get_ylim())
ax.set_title("Zoomed Raster")
ax.coastlines()

#### Controlling the resolution

You can control the resolution of the rasterization by adjusting the `pixel_ratio` parameter.
A value greater than 1 increases the resolution (sharpens the image), while a value less than 1 will result in a coarser rasterization.
The resolution also depends on what the figure's DPI setting is prior to calling {meth}`~uxarray.UxDataArray.to_raster`.

The `pixel_ratio` parameter can also be used with the standard HoloViz/Datashader-based plotting
(i.e. the {meth}`~uxarray.UxDataArray.plot` accessor; examples in [](plotting.ipynb)).

In [ ]:
pixel_ratios = [0.1, 0.5, 1, 4]

fig, axs = plt.subplots(
    len(pixel_ratios),
    1,
    subplot_kw={"projection": ccrs.Robinson()},
    figsize=(6, 8),
    constrained_layout=True,
    sharex=True,
    sharey=True,
)

axs.flat[0].set_extent((-20, 20, -5, 5))

for ax, pixel_ratio in zip(axs.flat, pixel_ratios):
    raster = uxds["bottomDepth"].to_raster(ax=ax, pixel_ratio=pixel_ratio)
    ax.imshow(
        raster, cmap="Blues", origin="lower", extent=ax.get_xlim() + ax.get_ylim()
    )
    ax.set_title(f"{pixel_ratio:.1f} Pixel Ratio", loc="left")
    ax.coastlines()

#### Reusing the pixel mapping

As we see below, this is helpful if you are planning to make multiple plots of the same scene, allowing the raster to be computed much more quickly after the first time.

Use `return_pixel_mapping=True` to get back the pixel mapping, and then pass it in the next time you call {meth}`~uxarray.UxDataArray.to_raster`.

In [ ]:
%%time

fig, ax = plt.subplots(
    figsize=(4, 2), subplot_kw={"projection": ccrs.Robinson()}, constrained_layout=True
)

ax.set_extent((-20, 20, -7, 7))

raster, pixel_mapping = uxds["bottomDepth"].to_raster(
    ax=ax, pixel_ratio=5, return_pixel_mapping=True
)
ax.imshow(raster, cmap="Blues", origin="lower", extent=ax.get_xlim() + ax.get_ylim())
ax.coastlines()

In [ ]:
pixel_mapping

In [ ]:
%%time

fig, ax = plt.subplots(
    figsize=(4, 2), subplot_kw={"projection": ccrs.Robinson()}, constrained_layout=True
)

ax.set_extent((-20, 20, -7, 7))

raster = uxds["bottomDepth"].to_raster(ax=ax, pixel_mapping=pixel_mapping)
ax.imshow(raster, cmap="Blues", origin="lower", extent=ax.get_xlim() + ax.get_ylim())
ax.coastlines()

### Viewing Contours with `ax.contour()` and `ax.contourf()`

You can use {meth}`ax.contour() <cartopy.mpl.geoaxes.GeoAxes.contour>` to draw projection-aware isolines and {meth}`ax.contourf() <cartopy.mpl.geoaxes.GeoAxes.contourf>` to shade between levels, specifying either a contour count or explicit thresholds.

```{warning}
The contours are generated on the raster image, not the unstructured grid geometries, which may create misleading results if not enough pixels were sampled.
```

In [ ]:
levels = [0, 2000, 4000, 6000]

In [ ]:
fig, axes = plt.subplots(
    2,
    1,
    subplot_kw={"projection": ccrs.Robinson()},
    constrained_layout=True,
    figsize=(9, 12),
)

ax1, ax2 = axes

ax1.set_global()
ax2.set_global()

ax1.coastlines()
ax2.coastlines()

raster = uxds["bottomDepth"].to_raster(ax=ax1)

# Contour Lines
ax1.contour(
    raster,
    cmap="Blues",
    origin="lower",
    extent=ax1.get_xlim() + ax1.get_ylim(),
    levels=levels,
)
ax1.set_title("Contour Lines")

# Filled Contours
ax2.contourf(
    raster,
    cmap="Blues",
    origin="lower",
    extent=ax2.get_xlim() + ax2.get_ylim(),
    levels=levels,
)
ax2.set_title("Filled Contours")

## Matplotlib Collections

Instead of directly sampling the unstructured grid, UXarray supports converting the grid into two {mod}`matplotlib.collections` classes: {class}`~matplotlib.collections.PolyCollection` and {class}`~matplotlib.collections.LineCollection`.

```{warning}
It is recommended to only use these collection-based plotting workflows if your grid is relatively small. For higher-resolution grids, directly rasterizing will almost always produce quicker results.
```

### Visualize Data with `PolyCollection`

To visualize face-centered data variables, you can convert a {class}`~uxarray.UxDataArray` into a {class}`~matplotlib.collections.PolyCollection`, which represents each face as a polygon, shaded by its corresponding data value.

In [ ]:
projection = ccrs.Robinson()
poly_collection = uxds["bottomDepth"].to_polycollection(projection=projection)

In [ ]:
poly_collection.set_cmap("Blues")

fig, ax = plt.subplots(
    1,
    1,
    facecolor="w",
    constrained_layout=True,
    subplot_kw=dict(projection=projection),
)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)

ax.add_collection(poly_collection)
ax.set_global()
plt.title("PolyCollection")

```{important}
By default, `periodic_elements` is set to `"exclude"`. 
```

To reduce the number of polygons in the collection, you can [subset](./subset) before converting.

In [ ]:
lon_bounds = (-50, 50)
lat_bounds = (-20, 20)
b = 15  # buffer for the selection so we can fill the plot area

poly_collection = (
    uxds["bottomDepth"]
    .subset.bounding_box(
        lon_bounds=(lon_bounds[0] - b, lon_bounds[1] + b),
        lat_bounds=(lat_bounds[0] - b, lat_bounds[1] + b),
    )
    .to_polycollection(projection=projection)
)

poly_collection.set_cmap("Blues")

fig, ax = plt.subplots(
    1,
    1,
    figsize=(7, 3.5),
    facecolor="w",
    constrained_layout=True,
    subplot_kw=dict(projection=projection),
)

ax.set_extent(lon_bounds + lat_bounds)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)

ax.add_collection(poly_collection)
plt.title("subset PolyCollection")

```{tip}
If the rendered polygons not perfectly aligning produces weird visual bugs, try using `poly_collection.set_edgecolor('face)` and adjusting with `poly_collection.set_linewidth()` if needed.
```

### Visualize Grid Topology


#### Using `PolyCollection`
To visualize the unstructured grid geometry, you can set the edge color on an existing `PolyCollection`.

In [ ]:
projection = ccrs.Robinson()
poly_collection = uxds["bottomDepth"].to_polycollection(projection=projection)

poly_collection.set_cmap("Blues")
poly_collection.set_edgecolor("black")

fig, ax = plt.subplots(
    1,
    1,
    facecolor="w",
    constrained_layout=True,
    subplot_kw=dict(projection=projection),
)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)

ax.add_collection(poly_collection)
ax.set_global()
plt.title("PolyCollection with set_edgecolor")

#### Using `LineCollection`
You can also convert a {class}`~uxarray.Grid` into a {class}`~matplotlib.collections.LineCollection`, which stores the edges of the unstructured grid.

In [ ]:
line_collection = uxds.uxgrid.to_linecollection(
    colors="black", linewidths=0.5, periodic_elements="split"
)

In [ ]:
fig, ax = plt.subplots(
    1,
    1,
    constrained_layout=True,
    subplot_kw={"projection": ccrs.Robinson()},
)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_collection(line_collection)
ax.set_global()
ax.set_title("LineCollection")
plt.show()